<a href="https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_02_2_pandas_cat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-558: Applications of Deep Neural Networks
**Module 2: Python for Machine Learning**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 2 Material

Main video lecture:

* Part 2.1: Introduction to Pandas [[Video]](https://www.youtube.com/watch?v=bN4UuCBdpZc&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_02_1_python_pandas.ipynb)
* **Part 2.2: Categorical Values** [[Video]](https://www.youtube.com/watch?v=4a1odDpG0Ho&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_02_2_pandas_cat.ipynb)
* Part 2.3: Grouping, Sorting, and Shuffling in Python Pandas [[Video]](https://www.youtube.com/watch?v=YS4wm5gD8DM&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_02_3_pandas_grouping.ipynb)
* Part 2.4: Using Apply and Map in Pandas for Keras [[Video]](https://www.youtube.com/watch?v=XNCEZ4WaPBY&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_02_4_pandas_functional.ipynb)
* Part 2.5: Feature Engineering in Pandas for Deep Learning in Keras [[Video]](https://www.youtube.com/watch?v=BWPTj4_Mi9E&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_02_5_pandas_features.ipynb)

# Google CoLab Instructions

The following code ensures that Google CoLab is running the correct version of TensorFlow.

In [1]:
try:
    %tensorflow_version 2.x
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Note: not using Google CoLab


# Part 2.2: Categorical and Continuous Values

Neural networks require their input to be a fixed number of columns.  This is very similar to spreadsheet data.  This input must be completely numeric.  

It is important to represent the data in a way that the neural network can train from it.  In class 6, we will see even more ways to preprocess data.  For now, we will look at several of the most basic ways to transform data for a neural network.

Before we look at specific ways to preprocess data, it is important to consider four basic types of data, as defined by [Stanley Smith Stevens](https://en.wikipedia.org/wiki/Stanley_Smith_Stevens).  These are commonly referred to as the [levels of measure](https://en.wikipedia.org/wiki/Level_of_measurement):

* Character Data (strings)
    * **Nominal** - Individual discrete items, no order. For example: color, zip code, shape.
    * **Ordinal** - Individual discrete items that can be ordered.  For example: grade level, job title, Starbucks(tm) coffee size (tall, vente, grande) 
* Numeric Data
    * **Interval** - Numeric values, no defined start.  For example, temperature.  You would never say "yesterday was twice as hot as today".
    * **Ratio** - Numeric values, clearly defined start.  For example, speed.  You would say that "The first car is going twice as fast as the second."

### Encoding Continuous Values

One common transformation is to normalize the inputs.  It is sometimes valuable to normalization numeric inputs to be put in a standard form so that two values can easily be compared.  Consider if a friend told you that he received a $10 discount.  Is this a good deal?  Maybe.  But the value is not normalized.  If your friend purchased a car, then the discount is not that good.  If your friend purchased dinner, this is a very good discount!

Percentages are a very common form of normalization.  If your friend tells you they got 10% off, we know that this is a better discount than 5%.  It does not matter how much the purchase price was.  One very common machine learning normalization is the Z-Score:

$z = \frac{x - \mu}{\sigma} $

To calculate the Z-Score you need to also calculate the mean($\mu$) and the standard deviation ($\sigma$).  The mean is calculated as follows:

$\mu = \bar{x} = \frac{x_1+x_2+\cdots +x_n}{n}$

The standard deviation is calculated as follows:

$\sigma = \sqrt{\frac{1}{N} \sum_{i=1}^N (x_i - \mu)^2}, {\rm \ \ where\ \ } \mu = \frac{1}{N} \sum_{i=1}^N x_i$

The following Python code replaces the mpg with a z-score.  Cars with average MPG will be near zero, above zero is above average, and below zero is below average.  Z-Scores above/below -3/3 are very rare, these are outliers.

In [2]:
import os
import pandas as pd
from scipy.stats import zscore

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv",
    na_values=['NA','?'])

df['mpg'] = zscore(df['mpg'])
display(df[0:5])

mpg  cylinders  displacement  horsepower  weight  acceleration  year  \
0 -0.706439          8         307.0       130.0    3504          12.0    70   
1 -1.090751          8         350.0       165.0    3693          11.5    70   
2 -0.706439          8         318.0       150.0    3436          11.0    70   
3 -0.962647          8         304.0       150.0    3433          12.0    70   
4 -0.834543          8         302.0       140.0    3449          10.5    70   

   origin                       name  
0       1  chevrolet chevelle malibu  
1       1          buick skylark 320  
2       1         plymouth satellite  
3       1              amc rebel sst  
4       1                ford torino

### Encoding Categorical Values as Dummies
The classic means of encoding categorical values is to make them dummy variables.  This is also called one-hot-encoding.  Consider the following data set.

In [3]:
import pandas as pd

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/jh-simple-dataset.csv",
    na_values=['NA','?'])

display(df[0:5])

id job area   income     aspect  subscriptions  dist_healthy  save_rate  \
0   1  vv    c  50876.0  13.100000              1      9.017895         35   
1   2  kd    c  60369.0  18.625000              2      7.766643         59   
2   3  pe    c  55126.0  34.766667              1      3.632069          6   
3   4  11    c  51690.0  15.808333              1      5.372942         16   
4   5  kl    d  28347.0  40.941667              3      3.822477         20   

   dist_unhealthy  age  pop_dense  retail_dense     crime product  
0       11.738935   49   0.885827      0.492126  0.071100       b  
1        6.805396   51   0.874016      0.342520  0.400809       c  
2       13.671772   44   0.944882      0.724409  0.207723       b  
3        4.333286   50   0.889764      0.444882  0.361216       b  
4        5.967121   38   0.744094      0.661417  0.068033       a

In [4]:
areas = list(df['area'].unique())
print(f'Number of areas: {len(areas)}')
print(f'Areas: {areas}')

Number of areas: 4
Areas: ['c', 'd', 'a', 'b']


There are four unique values in the areas column.  To encode these to dummy variables we would use four columns, each of which would represent one of the areas.  For each row, one column would have a value of one, the rest zeros.  This is why this type of encoding is sometimes called one-hot encoding.  The following code shows how you might encode the values "a" through "d".  The value A becomes [1,0,0,0] and the value B becomes [0,1,0,0].

In [5]:
dummies = pd.get_dummies(['a','b','c','d'],prefix='area')
print(dummies)

   area_a  area_b  area_c  area_d
0       1       0       0       0
1       0       1       0       0
2       0       0       1       0
3       0       0       0       1


To encode the "area" column, we use the following.

It is necessary to merge these dummies back into the data frame.  

In [6]:
dummies = pd.get_dummies(df['area'],prefix='area')
print(dummies[0:10]) # Just show the first 10

   area_a  area_b  area_c  area_d
0       0       0       1       0
1       0       0       1       0
2       0       0       1       0
3       0       0       1       0
4       0       0       0       1
5       0       0       1       0
6       0       0       0       1
7       1       0       0       0
8       0       0       1       0
9       1       0       0       0


In [7]:
df = pd.concat([df,dummies],axis=1)

Displaying select columns from the dataset we can see the dummy variables added.

In [8]:
display(df[0:10][['id','job','area','income','area_a',
                  'area_b','area_c','area_d']])

id job area   income  area_a  area_b  area_c  area_d
0   1  vv    c  50876.0       0       0       1       0
1   2  kd    c  60369.0       0       0       1       0
2   3  pe    c  55126.0       0       0       1       0
3   4  11    c  51690.0       0       0       1       0
4   5  kl    d  28347.0       0       0       0       1
5   6  e2    c  70854.0       0       0       1       0
6   7  kl    d  38726.0       0       0       0       1
7   8  nb    a  55162.0       1       0       0       0
8   9  al    c  67311.0       0       0       1       0
9  10  pe    a  63344.0       1       0       0       0

Usually, you will remove the original column ('area'), because it is the goal to get the dataframe to be entirely numeric for the neural network.

In [9]:
df.drop('area', axis=1, inplace=True)
display(df[0:10][['id','job','income','area_a',
                  'area_b','area_c','area_d']])

id job   income  area_a  area_b  area_c  area_d
0   1  vv  50876.0       0       0       1       0
1   2  kd  60369.0       0       0       1       0
2   3  pe  55126.0       0       0       1       0
3   4  11  51690.0       0       0       1       0
4   5  kl  28347.0       0       0       0       1
5   6  e2  70854.0       0       0       1       0
6   7  kl  38726.0       0       0       0       1
7   8  nb  55162.0       1       0       0       0
8   9  al  67311.0       0       0       1       0
9  10  pe  63344.0       1       0       0       0

### Target Encoding for Categoricals

Target encoding can sometimes increase the predictive power of a machine learning model.  However, it also greatly increases the risk of overfitting.  Because of this risk, care must be take if you are going to use this method.  It is a popular technique for Kaggle competitions.  

Generally, target encoding can only be used on a categorical feature when the output of the machine learning model is numeric (regression).

The concept of target encoding is actually very simple.  For each value 

In [10]:
# Create a small sample dataset
import pandas as pd
import numpy as np

np.random.seed(43)
df = pd.DataFrame({
    'cont_9': np.random.rand(10)*100,
    'cat_0': ['dog'] * 5 + ['cat'] * 5,
    'cat_1': ['wolf'] * 9 + ['tiger'] * 1,
    'y': [1, 0, 1, 1, 1, 1, 0, 0, 0, 0]
})

display(df)

cont_9 cat_0  cat_1  y
0  11.505457   dog   wolf  1
1  60.906654   dog   wolf  0
2  13.339096   dog   wolf  1
3  24.058962   dog   wolf  1
4  32.713906   dog   wolf  1
5  85.913749   cat   wolf  1
6  66.609021   cat   wolf  0
7  54.116221   cat   wolf  0
8   2.901382   cat   wolf  0
9  73.374830   cat  tiger  0

Rather than creating dummy variables for dog and cat, we would like to change it to a number. We could just use 0 for cat, 1 for dog.  However, we can encode more information than just that.  The simple 0 or 1 would also only work for one animal.  Consider what the mean target value is for cat and dog.

In [11]:
means0 = df.groupby('cat_0')['y'].mean().to_dict()
means0

{'cat': 0.2, 'dog': 0.8}

The danger is that we are now using the target value for training.  This will potentially overfit.  The possibility of overfitting is even greater if there are a small number of a particular category.  To prevent this from happening, we use a weighting factor.  The stronger the weight the more than categories with a small number of values will tend towards the overall average of y, which is calculated as follows.

In [12]:
df['y'].mean()

0.5

The complete function for target encoding is given here.

In [13]:
# Source: https://maxhalford.github.io/blog/target-encoding-done-the-right-way/
def calc_smooth_mean(df1, df2, cat_name, target, weight):
    # Compute the global mean
    mean = df[target].mean()

    # Compute the number of values and the mean of each group
    agg = df.groupby(cat_name)[target].agg(['count', 'mean'])
    counts = agg['count']
    means = agg['mean']

    # Compute the "smoothed" means
    smooth = (counts * means + weight * mean) / (counts + weight)

    # Replace each value by the according smoothed mean
    if df2 is None:
        return df1[cat_name].map(smooth)
    else:
        return df1[cat_name].map(smooth),df2[cat_name].map(smooth.to_dict())

The following code encodes these two categories.

In [14]:
WEIGHT = 5
df['cat_0_enc'] = calc_smooth_mean(df1=df, df2=None, cat_name='cat_0', target='y', weight=WEIGHT)
df['cat_1_enc'] = calc_smooth_mean(df1=df, df2=None, cat_name='cat_1', target='y', weight=WEIGHT)

In [15]:
display(df)

cont_9 cat_0  cat_1  y  cat_0_enc  cat_1_enc
0  11.505457   dog   wolf  1       0.65   0.535714
1  60.906654   dog   wolf  0       0.65   0.535714
2  13.339096   dog   wolf  1       0.65   0.535714
3  24.058962   dog   wolf  1       0.65   0.535714
4  32.713906   dog   wolf  1       0.65   0.535714
5  85.913749   cat   wolf  1       0.35   0.535714
6  66.609021   cat   wolf  0       0.35   0.535714
7  54.116221   cat   wolf  0       0.35   0.535714
8   2.901382   cat   wolf  0       0.35   0.535714
9  73.374830   cat  tiger  0       0.35   0.416667

### Encoding Categorical Values as Ordinal

Typically categoricals will be encoded as dummy variables.  However, there might be other techniques to convert categoricals to numeric. Any time there is an order to the categoricals, a number should be used.  Consider if you had a categorical that described the current education level of an individual.   

* Kindergarten (0)
* First Grade (1)
* Second Grade (2)
* Third Grade (3)
* Fourth Grade (4)
* Fifth Grade (5)
* Sixth Grade (6)
* Seventh Grade (7)
* Eighth Grade (8)
* High School Freshman (9)
* High School Sophomore (10)
* High School Junior (11)
* High School Senior (12)
* College Freshman (13)
* College Sophomore (14)
* College Junior (15)
* College Senior (16)
* Graduate Student (17)
* PhD Candidate (18)
* Doctorate (19)
* Post Doctorate (20)

The above list has 21 levels.  This would take 21 dummy variables. However, simply encoding this to dummies would lose the order information.  Perhaps the easiest approach would be to assign simply number them and assign the category a single number that is equal to the value in parenthesis above.  However, we might be able to do even better.  Graduate student is likely more than a year, so you might increase more than just one value.  